<a href="https://colab.research.google.com/github/dimitarpg13/langchain_tutorial/blob/main/langchain_tutorial/notebooks/runtime_config/ConfigurableLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with Runtime Configurations


We want to configure our graph when calling it. For example, we want to specify LLM or system prompt to use at runtime _without polluting the graph state with these parameters_.

Adding runtime configuration, invloves a) specifying a schema for the configuration, b) adding the configuration to the nodes or conditional edges and c) passing the configuration to the graph

In [9]:
%%capture --no-stderr
%pip install --quiet -U langgraph==0.6.0a2 langchain_core  langchain_openai langchain-anthropic dotenv

Specify the necessary keys

In [10]:
import os, getpass
from dotenv import load_dotenv

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

load_dotenv()

_set_env("OPENAI_API_KEY")

_set_env("ANTHROPIC_API_KEY")


## Speciyfing LLM at runtime

Below is a simple example which illustrates how to specify LLM at runtime using the Runtime Configuration functionality:

In [11]:
from dataclasses import dataclass

from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, StateGraph, START, MessagesState
from typing_extensions import TypedDict

from langchain.chat_models import init_chat_model
from langgraph.runtime import Runtime

1) Specify the context schema

In [12]:
@dataclass
class ContextSchema:
    model_provider: str = "anthropic"

MODELS = {
    "anthropic": init_chat_model("anthropic:claude-3-5-haiku-latest"),
    "openai": init_chat_model("openai:gpt-4.1-mini")
}



define `call_model` and construct the graph


In [13]:
def call_model(state: MessagesState, runtime: Runtime[ContextSchema]):
  model = MODELS[runtime.context.model_provider]
  response = model.invoke(state["messages"])
  return {"messages": [response]}

builder = StateGraph(MessagesState, context_schema=ContextSchema)
builder.add_node("model", call_model)
builder.add_edge(START, "model")
builder.add_edge("model", END)

graph = builder.compile()

User the graph with test input message

In [14]:
input_message = {"role": "user", "content": "hi"}

With no configuration, this graph uses default (Anthropic)

In [15]:
response_1 = graph.invoke({"messages": [input_message]})["messages"][-1]

AttributeError: 'NoneType' object has no attribute 'model_provider'

with configuration supplied it opens what is specified:

In [16]:
response_2 = graph.invoke({"messages": [input_message]}, config={"configurable": {"context": ContextSchema(model_provider="openai")}})["messages"][-1]
response_3 = graph.invoke({"messages": [input_message]}, config={"configurable": {"context": ContextSchema(model_provider="anthropic")}})["messages"][-1]

print(response_1.response_metadata["model_name"])
print(response_2.response_metadata["model_name"])
print(response_3.response_metadata["model_name"])

AttributeError: 'NoneType' object has no attribute 'model_provider'